# 基本数据处理

## 简单处理

- 使用固定长度的滑动窗口构造短序列
- 划分训练集/验证集/测试集

In [1]:
import numpy as np

from dataset.dataset import *

dataset = KrakowDataset()
raw_df = dataset.data.set_index(['sensor_index', 'UTC time'])

C:\Users\loganlin\Documents\Code\Python\Class\deeplearning2020\dataset\dataset.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_sensor['sensor_index'] = id2index[sensor_id]


下面是一段比较长的时间序列。

In [2]:
long_seq = raw_df.loc[2, '2017-01']['temperature'].dropna()
long_seq

UTC time
2017-01-01 00:00:00    0.264706
2017-01-01 01:00:00    0.250000
2017-01-01 02:00:00    0.250000
2017-01-01 03:00:00    0.250000
2017-01-01 04:00:00    0.250000
                         ...   
2017-01-31 19:00:00    0.294118
2017-01-31 20:00:00    0.294118
2017-01-31 21:00:00    0.279412
2017-01-31 22:00:00    0.294118
2017-01-31 23:00:00    0.279412
Name: temperature, Length: 691, dtype: float64

使用固定长度（如12）的滑动窗口，将其处理为几条短序列。

In [3]:
window_size = 12
short_seqs = []
for i in range(long_seq.shape[0] - window_size + 1):
    short_seqs.append(long_seq.iloc[i:i+window_size].tolist())
short_seqs = np.array(short_seqs)
print(short_seqs.shape)

(680, 12)


如果需要划分训练集/验证集/测试集，请首先对完整、有序的原始长序列按比例划分，再分别进行滑动窗口，而不是首先用滑动窗口生成多条短序列，再划分短序列。

In [4]:
train_set_proportion, val_set_proportion = 0.6, 0.2
total_len = long_seq.shape[0]
train_val_split = int(total_len * train_set_proportion)
val_test_split = int(total_len * (train_set_proportion + val_set_proportion))
train_seq, val_seq, test_seq = long_seq[:train_val_split],\
                               long_seq[train_val_split:val_test_split],\
                               long_seq[val_test_split:]

train_set = []
for i in range(train_seq.shape[0] - window_size):
    train_set.append(train_seq.iloc[i:i+window_size].tolist())
train_set = np.array(train_set)
print(train_set.shape)

(402, 12)


## 高级处理

- 固定时间跨度的滑动窗口
- 不等长序列的填充
- 不等长序列的打包

下面是一段长序列，然而，其时间轴并不规整，某些点之间空缺了几小时的数据。

In [5]:
long_seq = raw_df.loc[2, '2017-02']['temperature'].dropna()
long_seq

UTC time
2017-02-01 00:00:00    0.279412
2017-02-01 01:00:00    0.294118
2017-02-01 02:00:00    0.279412
2017-02-01 03:00:00    0.279412
2017-02-01 04:00:00    0.264706
                         ...   
2017-02-28 19:00:00    0.411765
2017-02-28 20:00:00    0.426471
2017-02-28 21:00:00    0.441176
2017-02-28 22:00:00    0.455882
2017-02-28 23:00:00    0.470588
Name: temperature, Length: 663, dtype: float64

### 填充与打包

我们可以通过使用固定时间跨度的滑动窗口使得生成的序列更合理。一般来说，这需要借助pandas DataFrame带时间戳的index。

In [6]:
window_size = 12  # (小时)
short_seqs = []
start_time, end_time = long_seq.index.min(), long_seq.index.max() - pd.Timedelta(window_size, 'h')
cur_time = start_time
while cur_time < end_time:
    short_seqs.append(long_seq.loc[cur_time:cur_time + pd.Timedelta(window_size-1, 'h')].tolist())
    cur_time += pd.Timedelta(1, 'h')

然而这会导致序列的长度不一致，无法处理为Tensor。

In [7]:
seq_lengths = [len(short_seq) for short_seq in short_seqs]
print('Minimum length:', min(seq_lengths))
print('Maximum length:', max(seq_lengths))

Minimum length: 3
Maximum length: 12


首先使用python自带的itertools中的函数，填充序列，使得所有序列的长度等于最长的序列。

In [8]:
from itertools import zip_longest

padded_seqs = np.array(list(zip_longest(*short_seqs, fillvalue=0))).transpose()
padded_seqs.shape

(659, 12)

在pytorch中使用时，可以使用函数将序列打包，使得pytorch能够合理地处理不等长的序列；即，被填充的部分不会实际输入到模型中。当然，这个函数需要手动输入每条序列的长度。

In [9]:
import torch
from torch.nn.utils.rnn import pack_padded_sequence

# 此序列可以直接输入torch封装好的RNN、GRU和LSTM。
packed_seqs = pack_padded_sequence(torch.tensor(padded_seqs), seq_lengths, 
                                   batch_first=True, enforce_sorted=False)

### 序列重采样

另一种方式是将原始长序列中缺失的时间戳补全。使用pandas的函数可以比较轻松实现这一功能。默认情况下，缺失的时间点会被填充nan。

In [10]:
start_time, end_time = long_seq.index.min(), long_seq.index.max() - pd.Timedelta(window_size, 'h')
full_index = pd.date_range(start_time, end_time, freq='h')
reindex_seq = long_seq.reindex(full_index)

随后我们可以使用插分函数，借助已有的数据，将空缺的数据补全。较常用的插分方法是线性插分。

In [11]:
inter_seq = reindex_seq.interpolate(method='linear', axis=0, limit=2, limit_direction='both')